# Reading Strategies

This is advanced feature meant to optimize message handling as the conversation grows. It is not necessary to understand this feature to use the library, but it will likely be required for production cases.

## The problems
LLMs have limited context windows, which means that they can only remember a certain number of previous messages. This can be a problem when the conversation grows and the model can't handle all of them. Additionally, models may get confused when the conversation is too long. And finally, keeping too much data can easily lead to ineffeciency (especially when running agents remotely)

## Solution

The solution is to use reading strategies. Reading strategies are a way to tell the model which messages to keep and which to discard. This way, the model can focus on the most relevant messages and ignore the rest. This can be done by implementing a `ReadingStrategy` class and passing it to an `Agent` or `Team`.

**NOTE** that the original conversation is NOT modified. The reading strategy is only used to filter the messages that the model sees at any given round.

## Available strategies

The library provides a few reading strategies that can be used out of the box. They are:

- `AllMessagesStrategy`: keeps all messages. Default one.
- `LastNMessagesStrategy`: keeps the last N messages.
- `TopKLastNMessagesStrategy`: keeps the top K messages and the last N messages.
- `SummarizeMessagesStrategy`: returns a single message that summarizes the conversation.
- `PipelineConversationReadingStrategy`: allows to chain multiple strategies.

## See also

Complemenary to reading strategies, there are also _update strategies_ that allow to change how agents update the conversation history. This is useful when you want to keep the conversation history clean and avoid duplicates. See the [update strategies notebook](update_strategies.ipynb) for more information.

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Add the parent directory to sys.path
import sys, os
sys.path.append(os.path.abspath(os.path.join('../vanilla_aiagents')))

from vanilla_aiagents.llm import AzureOpenAILLM
from dotenv import load_dotenv

load_dotenv(override=True)

In [3]:
llm = AzureOpenAILLM({
    "azure_deployment": os.getenv("AZURE_OPENAI_MODEL"),
    "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.getenv("AZURE_OPENAI_KEY"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
})

# Set logging to debug for Agent, User and Workflow
import logging

# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.INFO)
logging.getLogger("vanilla_aiagents.agent").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.workflow").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.team").setLevel(logging.DEBUG)
logging.getLogger("vanilla_aiagents.llm").setLevel(logging.DEBUG)

In [4]:
from vanilla_aiagents.conversation import Conversation, SummarizeMessagesStrategy
conversation = Conversation(messages=[
            {"role": "system", "content": ""},
            {"role": "assistant","name": "agent", "content": "Hello! How can I help you today?"},
            {"role": "user","name": "user", "content": "Hi! Can you help me with capital cities?"},
            {"role": "assistant","name": "agent", "content": "Sure! Which capital you want to learn about?"},
            {"role": "user","name": "user", "content": "Which is the capital of Italy?"},
            {"role": "assistant","name": "agent", "content": "The capital of Italy is Rome."},
            {"role": "user","name": "user", "content": "And the capital of France?"},
            {"role": "assistant","name": "agent", "content": "The capital of France is Paris."},
            {"role": "user","name": "user", "content": "Thank you!"},
            {"role": "assistant","name": "agent", "content": "You're welcome!"},
        ])

In [ ]:

strategy = SummarizeMessagesStrategy(llm=llm, system_prompt="Summarize the conversation, highlighting the key points in a bullet list")
result = strategy.get_messages(conversation)
result[0]["content"]